# EDA tests

Trying a few EDA tools using the basic housing regression dataset. Mostly it is about visualizing the data, also with the help of some automated analysis tools. ##

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from tqdm.notebook import tqdm
from autoviz.classify_method import data_cleaning_suggestions, data_suggestions
from ydata_profiling import ProfileReport

%matplotlib inline

In [ ]:
%cd ../data

In [ ]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

# Goal for Housing Dataset

This data is from the Kaggle [Housing competition](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques) for practicing regression problems.
The goal here is to use these basic housing features to predict house price.

In [ ]:
df_train.head()

In [ ]:
#list + remove should preserve order
data_columns = list(df_train.columns)
data_columns.remove("Id")

In [ ]:
!ls

## Data Descriptions

The file "data_descriptions.txt" should contain descriptions for the columns in the dataset. Building a simple mapping here for column and value identifiers to their names and descriptions.

In [ ]:
with open("data_description.txt") as f:
    lines = f.readlines()
    #print(lines)
    

In [ ]:
lines[:5]

The data_descriptions.txt file contains a description of each column, and for the categorical ones a mapping of values ("20", "30", "40" in above example) to their descriptions. Some columns lack a descriptions, I add those separately.

In [ ]:
column_name = None
column_descriptions = {}
key_descriptions = None

# not all column actually have a description in the file, so need to add manually
# the way to identify is to try them all and collect which ones are missing
def add_missing_col_desc(col_name, col_desc):
    column_descriptions[col_name] = {}
    column_descriptions[col_name]["description"] = col_desc
    column_descriptions[col_name]["keys"] = {}

add_missing_col_desc("SalePrice", "Target variable")
    
for line in tqdm(lines):
    stripped = line.strip()
    if len(stripped) > 0 and not line.startswith(" "):
        # this branch happens when a new column description starts
        parts = stripped.split(":")
        column_name = parts[0]
        column_description = parts[1].strip()
        key_descriptions = {}
        column_descriptions[column_name] = {}
        column_descriptions[column_name]["keys"] = key_descriptions
        column_descriptions[column_name]["description"] = column_description
        print(f"column: {column_name} = {column_description}")
    else:
        # this branch happens when a column name is identified and its key is processed
        parts = stripped.split("\t")
        if len(parts) > 1:
            key = parts[0].strip()
            description = parts[1].strip()
            key_descriptions[key] = description
            #print(parts)

In [ ]:
# for some reason these two are missing from the descriptions file, so had to make it up
add_missing_col_desc("BedroomAbvGr", "Number of bedrooms above ground")
add_missing_col_desc("KitchenAbvGr", "Number of kitchens above ground")


# Visualizing Valuespaces

First, I try to build a chart describing value space for one variable. Later once this works, it is easier to expand to other variables (columns) as well.

In [ ]:
col_name = data_columns[0]
x = df_train[col_name]

value_counts = x.value_counts().sort_index()
keys = value_counts.index
keys = [str(key) for key in keys]

fig, ax = plt.subplots()
bars = ax.bar(keys, value_counts)

title = column_descriptions[col_name]["description"]
title = f"{col_name}: {title}"
ax.set_title(title, fontsize=12, fontweight='bold')

keys = column_descriptions[col_name]["keys"]
keys_txt = ""
for key in keys:
    keys_txt += f"{key}: {keys[key]}\n"
ax.annotate(keys_txt, (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top')

ax.bar_label(bars)

In [ ]:
import json

# printing all column descriptions makes it a too long list for the lazy reader, 
# so i stick with 2 column_descriptions
print(json.dumps(column_descriptions[data_columns[0]], indent=4))
print(json.dumps(column_descriptions[data_columns[1]], indent=4))


# Fixing Typos in Data

Some of the value descriptions in data_description.txt file do not match the actual column/key names in the data file itself. Because of typos or different form of writing. This replacement changes that, to make them match:

In [ ]:
replacements = {"C (all)": 'C', 
                "NAmes": "Names", 
                "Duplx": "Duplex",
                "2fmCon": "2FmCon",
                "Twnhs": "TwnhsI",
                "CmentBd": "CemntBd",
                "Brk Cmn": "BrkComm",
                "Wd Shng": "WdShing",
               }

def process_values(col_name, values):
    if col_name == "MSSubClass":
        return [str(v) for v in values]
    # if it is in the above replacemens table use it, else leave it as is
    values = [replacements[v] if v in replacements else v for v in values]
    return values

def process_keys(col_name, keys):
    if col_name == "OverallQual" or col_name == "OverallCond":
        return [int(v) for v in keys]
    keys = [replacements[v] if v in replacements else v for v in keys ]
    return keys
    

# Split Variables by Data Type

Now to collect categorical vs numerical columns. Here numerical just means continuous variables. After this it is possible to process different types of data in different ways based on type.

In [ ]:
numeric_cols = ["SalePrice"]
cat_num_cols = []
cat_cols = []

for col_name in data_columns:
    if col_name not in column_descriptions:
        print(f"WARN: column not in descriptions: {col_name}")
        # here i added BedroomAbvGr and KitchenAbvGr to descriptions as they were missing
        continue
#    print(col_name)
#    print(column_descriptions[col_name]["keys"])
    keys = column_descriptions[col_name]["keys"].keys()
    keys = process_keys(col_name, keys)
    key_count = len(keys)
    if key_count == 0:
        print(f"{col_name}: zero keys, assuming numeric (continuous) value")
        if df_train[col_name].nunique() < 20:
            # some variables actually have very few unique values, so will treat them
            # in some ways as categorical later (visualization type at least)
            cat_num_cols.append(col_name)
        else:
            # these are the actual variables assumed to have continuous values
            numeric_cols.append(col_name)
        continue
    else:
        cat_cols.append(col_name)


In [ ]:
# method used to check value space of missing variable
# BedroomAvgGr seems to indicate number of bedrooms above ground by named values
# while it might not be categorical as such, visualizing it using techniques for 
# datasets with a few distinct values later seems more useful than line charts for continous
# mostly becaues there number of value options is so small, and in a way categorical even
df_train["BedroomAbvGr"].value_counts()

In [ ]:
cat_num_cols

In [ ]:
cat_cols

# Remove Columns with Single Value

Sometimes there are columns with a single value, this checks for such columns and removes them as features is they exist. Should check for NaN values separately perhaps.

In [ ]:
cols_to_drop = []

for col in df_train.columns:
    #print(df_train[col].nunique())
    if df_train[col].nunique() == 1:
        print("match")
        cols_to_drop.append(col)

cols_to_drop

In [ ]:
# this variable had a different value for missing/empty variables, so keeping in line with that
df_train["MasVnrType"] = df_train["MasVnrType"].fillna("None")
# and filling the rest with "NA"
df_train[cat_cols] = df_train[cat_cols].fillna("NA")

# Check for Mismatch in Expected vs Found Values

The data description text file has some values described for the categorical variables. Sometimes these differ from the ones found in the actual data. The following code builds two sets, one listing the values found in the csv file but not in the description text file. And one listing the values found in the text file but not in the csv data.

## Collect Mismatched Values

In [ ]:
# key = column name, value=list of values found in text file but not in data
unknown_keys_dict = {}
# key = column name, value=list of values found in data but not in text file
unknown_values_dict = {}
# key = column name, value = number of unique values for it found in data
cat_value_counts = {}

for col_name in cat_cols:
    keys = column_descriptions[col_name]["keys"].keys()
    keys = process_keys(col_name, keys)
    key_count = len(keys)
    unique_values = df_train[col_name].unique()
    cat_value_counts[col_name] = df_train[col_name].value_counts()
    unique_values = process_values(col_name, unique_values)
    unique_count = len(unique_values)
    unknown_keys = set(keys) - set(unique_values)
    unknown_values = set(unique_values) - set(keys)
    unknown_keys_dict[col_name] = unknown_keys
    unknown_values_dict[col_name] = unknown_values


## Values in Descriptions but not in Data

Print all keys and their descriptions found in text file but not in data.

In [ ]:
for col_name in unknown_keys_dict:
    unknown_keys = unknown_keys_dict[col_name]
    col_keys = column_descriptions[col_name]["keys"]
    if len(unknown_keys) > 0:
        # print names of columns with keys found in description text file but not in data
        print(f"{col_name}:")
        for key in unknown_keys:
            # print the unknown values
            print(f"- {key}: {col_keys[str(key)]}")
            print(cat_value_counts[col_name].dtype)
            if cat_value_counts[col_name].index.is_integer():
                key = int(key)
            # later on cat_value_counts may be used for visualization, so need to fill it
            cat_value_counts[col_name][key] = 0
        # print counts for found values in data, to compare why it might be missing
        print(df_train[col_name].value_counts())
        print("----------")
        print(cat_value_counts[col_name])
        print()
            

## Values in Data but no in Descriptions

Print values found in data but not in the descriptions text file.
Here Eletrical has NA value but this is not in the text file.
Perhaps due to earlier filling empty slots with NA. In any case, 
the empty (nan) value would not be there either

In [ ]:
for col_name in unknown_values_dict:
    unknown_values = unknown_values_dict[col_name]
    col_keys = column_descriptions[col_name]["keys"]
    if len(unknown_values) > 0:
        print(f"{col_name}: {unknown_values}")
        print(col_keys)
        print(df_train[col_name].value_counts())
            

# Visualize Categorical vs Continous Variables

Now to visualize the different column values using bar charts for the categorical values and line charts for the continous values. Along with the descriptions from the text file. First try with just 2 columns to see how it might work:

In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(12,4))

for p in range(2):
    col_name = data_columns[p]
    x = df_train[col_name]

    value_counts = cat_value_counts[col_name].sort_index()
    keys = value_counts.index
    keys = [str(key) for key in keys]

    ax = axes[p]
    bars = ax.bar(keys, value_counts)

    title = column_descriptions[col_name]["description"]
    title = f"{col_name}:"
    print(title)
    ax.set_title(title, fontsize=8, fontweight='bold')

    keys = column_descriptions[col_name]["keys"]
    keys_txt = ""
    for key in keys:
        keys_txt += f"{key}: {keys[key]}\n"
    ax.annotate(keys_txt, (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top')

    # add count labels on top of the bars
    ax.bar_label(bars)

## Categorical Columns

Next to visualize all categorical / low count numerical columns:

In [ ]:
# The following loop should be number of cat cols / 2 as 2 charts per row
len(cat_value_counts)

In [ ]:
# 2 charts per row, so 23*2=46
for row in range(0,23):
    fig, axes = plt.subplots(1, 2, figsize=(12,4))

    for p in range(2):
#        col_name = data_columns[row*2+p]
        col_name = list(cat_value_counts.keys())[row*2+p]
        x = df_train[col_name]

    #    value_counts = x.value_counts().sort_index()
        #print(cat_value_counts[col_name].index)
        value_counts = cat_value_counts[col_name].sort_index()
        #print(value_counts)
        keys = value_counts.index
        keys = [str(key) for key in keys]

        ax = axes[p]
        bars = ax.bar(keys, value_counts)

        title_description = column_descriptions[col_name]["description"]
        # the title + description was too long and charts became messy
    #    title = f"{col_name}: {title}"
        title = f"{col_name}:"
        print(title+" "+title_description)
        ax.set_title(title, fontsize=12, fontweight='bold')

        keys = column_descriptions[col_name]["keys"]
        keys_txt = ""
        for key in keys:
            keys_txt += f"{key}: {keys[key]}\n"
        ax.annotate(keys_txt, (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top')

        ax.bar_label(bars)

In the above it shows why I wanted to add 0 as a value for those keys that had no value in data. This way the descriptions vs bars are easier to map together as there is always some value in both.

The charts above also highlight how some columns might have very few distinct values, such as Utilities only having 2 classes and all but one value in the AllPublic category. Sometimes it can be useful information though, for example, to know the house has no sewage. So why not keep it? Well depends on the use case I guess. 

There are also many others with a bit more values, such as Street pavement type of PoolQC. One might combine these into features such as HasPool if the separate categories are very small. But again, I guess depends on the use case.

# Outlier Analysis

Before visualizing the continous variables, I will try two basic outlier detection methods. These are the [standard deviation - based method](https://stats.stackexchange.com/questions/575483/can-i-remove-sample-outliers-using-standard-deviation), and the [interquantile range - based method](https://math.stackexchange.com/questions/966331/why-john-tukey-set-1-5-iqr-to-detect-outliers-instead-of-1-or-2).

Both of these methods are run, and the results are stored in their own dataframe each for later analysis.

Looking at the results and the later visualizations, I believe it would make much more sense to look at the data and figure out what kind of operations to run on it first. In many cases here, the outlier removal does not seem to make much sense. I guess that is a good point to remember, if only I souldn't forget it by tomorrow anyway.

In [ ]:
outlier_thresholds_high_std = {}
outlier_thresholds_low_std = {}
outlier_thresholds_high_tukey = {}
outlier_thresholds_low_tukey = {}

#assuming the data is normally distributed, this removes outliers using the "three sigma" rule.
#that is 3*std from mean is expected to contain the 0.03% of smallest/highest values.
#those are replaced with the min/mam here (actually just max in this implementation)
def remove_outliers_normal(df, col):
    df = df.copy()
    upper = df[col].mean()+3*df[col].std()
    lower = df[col].mean()-3*df[col].std()
    #even variable outlier_thresholds does not exist in this code, since this method variant was not used by me in the end
    #however, should be trivial to fix both min/max and thresholding
    if col in outlier_thresholds_high_std:
        upper = outlier_thresholds_high_std[col]
        lower = outlier_thresholds_low_std[col]
    else:
        outlier_thresholds_high_std[col] = upper
        outlier_thresholds_low_std[col] = lower
        
    high_mask = df[col] > upper
    low_mask = df[col] < lower
    df.loc[high_mask, col] = upper
    df.loc[low_mask, col] = lower
    print(f"col: {col}, hight std: {sum(high_mask)}, low std: {sum(low_mask)}")
    return df
    
#tukey outlier removal should work for any distribution, not just normal.
#the normal version above is just perhaps more focused for normal distributions
def remove_outliers_tukey(df, col):
    df = df.copy()
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3-q1
    upper = q3 + iqr*2 #tukey default uses 1.5 multiplier, using *2 here to get more extreme outliers
    lower = q1 - iqr*2 #*3 gave std up to 5.5x, lets see 2*
    # if the values are really small, it messes this method up. so this just addresses that
    if iqr < 1:
        print(f"very small IQR for {col}, defaulting to last quantile or 1% and 99% outside capping")
        upper = df[col].quantile(0.99)
        lower = df[col].quantile(0.01)

    if col in outlier_thresholds_high_tukey:
        upper = outlier_thresholds_high_tukey[col]
        lower = outlier_thresholds_low_tukey[col]
    else:
        outlier_thresholds_high_tukey[col] = upper
        outlier_thresholds_low_tukey[col] = lower

    high_mask = df[col] > upper
    low_mask = df[col] < lower
    print(f"col: {col}, q1: {q1}, q3: {q3}, iqr: {iqr}, hight tukey: {sum(high_mask)}, low tukey: {sum(low_mask)}")
    
    #print(sum(mask))
    df.loc[high_mask, col] = upper

    #print(sum(mask))
    df.loc[low_mask, col] = lower
    return df

In [ ]:
def remove_df_outliers(df):
    df_tukey = df
    df_std = df
    for col in tqdm(numeric_cols):
        #note that in its current implementation remove_outliers_tukey does not work across multiple dataframes
        #if the outlier threshold is updated by a latter dataframe, the earlier ones used a different one
        df_tukey = remove_outliers_tukey(df_tukey, col)
        df_std = remove_outliers_normal(df_std, col)
    return df_tukey, df_std
        

In [ ]:
df_tukey, df_std = remove_df_outliers(df_train)

Convert the outlier thresholds for the different methods into a dataframe for analysis.

In [ ]:
df_thresholds = pd.DataFrame()
df_thresholds["column_name"] = numeric_cols

tukey_lows = []
tukey_highs = []
std_lows = []
std_highs = []

for col in numeric_cols:
    tukey_lows.append(outlier_thresholds_low_tukey[col])
    tukey_highs.append(outlier_thresholds_high_tukey[col])
    std_lows.append(outlier_thresholds_low_std[col])
    std_highs.append(outlier_thresholds_high_std[col])
    
df_thresholds["tukey_low"] = tukey_lows
df_thresholds["tukey_high"] = tukey_highs
df_thresholds["std_low"] = std_lows
df_thresholds["std_high"] = std_highs
df_thresholds

## Look at Saleprice Distribution before Outlier Removal

In [ ]:
df_train["SalePrice"].value_counts().sort_index()


In [ ]:
df_tukey["SalePrice"].value_counts().sort_index()

In [ ]:
df_std["SalePrice"].value_counts().sort_index()

## Binning Continous Data for Plotting

Now to plot the saleprice. First, collect the bin ranges to plot:

In [ ]:
low = df_train["SalePrice"].min()
high = df_train["SalePrice"].max()
print(low)
print(high)
bin_ranges = np.arange(0, 800001, 50000)
bin_ranges

Matplotlib with bar_label makes the labeling of bars easier:

In [ ]:
bins = pd.cut(df_train["SalePrice"], bins=bin_ranges).value_counts()
bins = bins.sort_index()
bins = pd.DataFrame(data=bins, index=bin_ranges, columns=["SalePrice"])
ax = bins.plot(kind="bar", figsize=(12, 8), width=0.9)

# annotate
ax.bar_label(ax.containers[0], label_type='edge')

Seaborn plots can be a little more pleasant looking, but labeling the bars seems difficult:

In [ ]:
sb.set_palette("deep")

ax = sb.displot(df_train['SalePrice'].values,
             bins=bin_ranges,
             kde=True, aspect=2) #defaults to height = 5, aspect = 1

# Plotting Continous Variables

Earlier (above) I plotted the categorical columns. And above tried the two outlier methods. Now for plotting the continous variables and their values, while also comparing the "raw" data vs the two outlier approaches (std and Tukey):

In [ ]:
def plot_dataframes(df1, df2, df3, title1, title2, title3):
    figs_per_row = 3
    fig_keys = numeric_cols
    row_count = len(fig_keys)
    fig_height = 5*row_count
    fig, axes = plt.subplots(row_count, figs_per_row, figsize=(12,fig_height))
    fig.subplots_adjust(hspace=0.5)

    dfs = [df1, df2, df3]
    titles = [title1, title2, title3]
    for p in range(len(numeric_cols)):
        #col_name = data_columns[p]
        col_name = numeric_cols[p]
        for df_id in range(0,3):
            df = dfs[df_id]
            x = df[col_name]

            #value_counts = x.value_counts()#.sort_index()
            value_counts = df[col_name].value_counts().sort_index()
            print(col_name)

            p_y = p
            ax = axes[p_y, df_id]
            if len(value_counts) < 20:
                keys = value_counts.index
                keys = [str(key) for key in keys]
                bars = ax.bar(keys, value_counts)
                ax.bar_label(bars)
            else:
                lines = ax.plot(value_counts)

            description = column_descriptions[col_name]["description"]
            desc_title = f"{col_name}: {description}"
            words = desc_title.split()
            new_desc_title = ""
            new_line = ""
            for word in words:
                if len(new_line) > 50:
                    new_desc_title += new_line + "\n"
                    new_line = ""
                else:
                    new_line += word + " "
            if len(new_line.strip()) > 0: 
                new_desc_title += new_line + "\n"
            desc_title = new_desc_title

            title = f"{col_name} ({titles[df_id]}):"
            print(title)
            ax.set_title(title, fontsize=12, fontweight='bold')

            ax.annotate(desc_title, (0,0), (0, -20), xycoords='axes fraction', textcoords='offset points', va='top')


In the following figures, the leftmost is a plot of the 'raw' data, the middle is the tukey processed outlier removal, and rightmost is the std processed outlier removal. If the Tukey or STD versions have a high spike on the right, this is typically from capping the long right-side tail.

In [ ]:
plot_dataframes(df_train, df_tukey, df_std, "base", "tukey", "std")

The zero values also show as high spikes on the left side, due to not having some property in the house. For example, MasVnrArea has a spike of about 800+ houses on the very left. Most likely due to most houses not having any such masonry (area size is 0).

Overall, my impression from the above side-by-side charts is that the outlier removal here is not really useful. While much of the data has a long tail, the spike on the right shows that quite many values had to be capped. And those larger values in house properties may well be useful information.

However, the outlier removals above do make the overall distribution more visible in many cases. So for such exploration purposes at least it seems useful. But in general, perhaps better to study the data bit before training any model on blindly capped outliers.

## Cleanup Suggestions after Outlier Removal

With correlations taken care of, let's look at what the data cleaning package suggest to do for cleaning up the data in the differently processed dataframes:

In [ ]:
data_cleaning_suggestions(df_train)

In [ ]:
data_cleaning_suggestions(df_tukey)

In [ ]:
data_cleaning_suggestions(df_std)

Some of the above look a little strange. For example BsmtFinSF2 and EnclosedPorch both have suggestions to remove outliers in the Tukey version, after the Tukey was all about removing outliers in the first place. And this suggestions is not in there for the original, uncapped version of the same data. The STD version of the data has even more of such suggestions. 

I would expect the tool to have rather suggested those for the original data, not the Tukey or STD outlier capped versions. Possibly something in how it translates the distribution, since the in the modified ones there are more values clusted at the right edge. Just a guess. 

But again, I would try to remember to check the data myself in detail rather than use only these automated tools, although they are useful aides.

## ProfileReports into HTML files

This actually writes some data summary reports into separate files. They seemed quite useful when looking at the generated files, for getting an overview. But harder to visualize in a notebook.

In [ ]:
report = ProfileReport(df_train,title='All features report')
report.to_file(output_file='Numeric_Feature_EDA.html')

# Correlations Check

Another common topic to check is to see correlations between different variables. As this is such as common topic, there is good support for calculating the correlations and visualizing them. First a correlation heatmap using Seaborn:

In [ ]:
corr = df_train[numeric_cols].corr()
#change annot=False to annot=True to get correlation values in the plot
#in this case it is just a bit too crowded after adding those
sb.heatmap(corr, cmap="Blues", annot=False)

And the same correlation values in dataframe, numerical format:

In [ ]:
corr

For a bit more detailed exploration of correlations, it is useful to be able to find the most correlated pairs, both negatively and positively. And why not the least correlated pairs.. 

These functions allow doing that:

In [ ]:
#https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas

# redundant pairs removes duplicate pairs, since typically correlation matrices show
# both the a->b and b->a correlation.

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

# this provides the list of top correlations, either positive, negative, or both ways (absolute)
# or even lear correlated (lowest absolute values)
def get_top_abs_correlations(df, n=5, asc=False, absolute=True):
    au_corr = df.corr().abs().unstack()
    if not absolute:
        #in case we want to get highest negative value
        au_corr = df.corr().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=asc)
    return au_corr[0:n]



Lets check the results of running the above functions:

In [ ]:
#these are as far from each other as possible
print("Top Absolute Un-Correlations")
print(get_top_abs_correlations(df_train[numeric_cols], 5, True))

# Plot Highest Correlations

In [ ]:
# the most correlated pairs, positively or negatively
print("Top Absolute Correlations")
highest_corrs = get_top_abs_correlations(df_train[numeric_cols], 5, False)
highest_corrs

In [ ]:
highest_corrs.index

Collect the actual column names for the highest correlations, for correlation plotting after:

In [ ]:
cols = [x[0] for x in highest_corrs.index]
cols += [x[1] for x in highest_corrs.index]
cols = list(set(cols))
cols

Pairplots, plotting all variables against each other to see a form of a correlation matrix. First using seaborn:

In [ ]:
sb.pairplot(df_train[cols], hue="YearBuilt")
#plt.show()

The charts above look strange since the scatterplots seems compressed on x-axis. It appears this is due to the KDE plot on diagonal using the same x-axis and having a much wider spread.
Changing the diagonal to histograms syncs the x-axis, makes the scatters more readable. I find the histogram easier to read than KDE anyway:

In [ ]:
g = sb.PairGrid(df_train[cols], diag_sharey=False)

g.map_lower(sns.scatterplot, s=50, hue=df_train["YearBuilt"])
g.map_diag(sns.histplot, kde=False)
#use a smaller plot size on the upper size, just to see the size parameter effect
g.map_upper(sns.scatterplot, s=10, hue=df_train["YearBuilt"])

g.fig.subplots_adjust(wspace=0, hspace=0)

plt.show()

The 3 bottom rows in these charts show a single high outlier for each of TotalBsmtSD, GrLivArea, and 1stFlrSF. Instead of blindly shooting with STD or Tukey capping, this seems a more realistic outlier to consider.

But from the above Seaborn charts it is hard to directly see what this value is. I could just filter the highest numbers for the above 3 columns, but interactively seeing it from the graph would be nice. 

While I did not find a way to enable such interaction in Seaborn, Plotly provides a simple way:

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(df_train[cols], width=1200, height=1200)
fig.show()

In the above Plotly charts, hovering over each dot with the mouse will display a tooltip with the values it is plotted from. For example, the outlier on TotalBsmtSF shows a value of 6110.
Filtering this value from the dataframe shows it is the same value that has the outlier values for most of the big outliers in the chart above (just mouse over the dots for columns):

In [ ]:

with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_train[df_train["TotalBsmtSF"] == 6110])

In [ ]:
df_train[df_train["TotalBsmtSF"] == 6110][["TotalBsmtSF", "2ndFlrSF", "GrLivArea"]]

Try to remove the manually identified outlier and plot again:

In [ ]:
df_train_2 = df_train[df_train["TotalBsmtSF"] != 6110]
df_train_2.shape

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(df_train_2[cols], width=1200, height=1200)
fig.show()

Looks much better now, showing the overall shape more clearly.

Now to see the "automated" Tukey and STD removed plots,

## Tukey Processed Plots

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(df_tukey[cols], width=1200, height=1200)
fig.show()

## STD Processed Plots

In [ ]:
import plotly.express as px

fig = px.scatter_matrix(df_std[cols], width=1200, height=1200)
fig.show()

The diagonal typically separates the two copies of the pairs in the above plots. That is, each pair is there twice. With Seaborn it is possible to add correlation coefficients into the plot, and to drop the duplicate plots if wanted:

In [ ]:
def corrfunc(x, y, **kws):
    #https://stackoverflow.com/questions/48591713/pearson-correlation-and-nan-values
    good = ~np.logical_or(np.isnan(x), np.isnan(y))
    r, _ = stats.pearsonr(x[good], y[good])
    annotation = f"{r:.2f}"
    ax = plt.gca()
    ax.annotate(f"r = {annotation}", fontsize=25,
                xy=(.1, .9), xycoords=ax.transAxes)


Plotting actual correlation values:

In [ ]:
# using pairgrid vs pairplot allows calculating and plotting correlation?
g = sb.PairGrid(df_train_2[cols], diag_sharey=False)

g.map_lower(sns.scatterplot, s=50, hue=df_train_2["YearBuilt"])
g.map_diag(sns.histplot, kde=True)
g.map_lower(corrfunc) # add also correlation coefficient annotations to lower half plots

g.fig.subplots_adjust(wspace=0, hspace=0)

plt.show()

Above plots show the actual Pearson correlation value now for the two variables of concern. 

Some of the data structure is quite well visible in these plots. For example:
- The 2ndFlrSF is zero in many cases, leading to a vertical line in the second column from the left. Because many houses don't have a second floor.
- In column descriptions GrLivArea is described as the living area above ground (see below). Thus, the 1stFlrSF vs GrLivArea has a diagonal line. The total living area above ground cannot be less than 1stFlrSF, and is only higher in cases where the house has a 2nd floor. Hence the diagonal line.
- The 1stFlrSF vs TotalBsmtSF has another diagonal line, with a few dots above it only. Because in very few cases the basement is larger than the living area (1st floor) above it. More commonly the size is the same or less.
- GarageYrBlt vs YearBuilt has a diagonal line with a few dots just very little below it. Because there might be a few cases where the garage finished the previous year before finishing the house. But appears never did someone build a garage and wait 10 years to build the house. Makes sense.
- YearRemodAdd is described as being the same as construction date if no remodelling done. However, this change does not seem to apply before 1950 or there is something else at hand, referring to the diagonal line starting from around 1950 for YearRemodAdd vs YearBuilt and GarageYrBlt. Also the yearRemodAdd row itself shows its value starting from 1950+ so apparently before that it is always zero.

In [ ]:
column_descriptions["GrLivArea"]

In [ ]:
column_descriptions["YearRemodAdd"]

Now to add the top plot, it shows the relations different in a way of A->B vs B->A, so might be helpful at time to capture visually some relations better.

In [ ]:
#sb.pairplot(df_tukey[cols], hue="YearBuilt")

g = sb.PairGrid(df_tukey[cols], diag_sharey=False)

#https://medium.com/@morganjonesartist/color-guide-to-seaborn-palettes-da849406d44f
g.map_lower(sns.scatterplot, s=50, hue=df_tukey["YearBuilt"])
g.map_diag(sns.histplot, kde=False)
g.map_upper(sns.scatterplot, s=10, hue=df_tukey["YearBuilt"])
g.map_lower(corrfunc)

g.fig.subplots_adjust(wspace=0, hspace=0)

plt.show()

## Tighten The Plot Margins

This is what I tried initially to get the compressed looking scatters to widen. But it just removes the small white space between the charts. Might be useful still, sometimes.

In [ ]:
import matplotlib as mpl

orig_xm = mpl.rcParams['axes.xmargin']
orig_ym = mpl.rcParams['axes.ymargin']
print(orig_xm)
print(orig_ym)

In [ ]:
mpl.rcParams['axes.xmargin'] = 0.01
mpl.rcParams['axes.ymargin'] = 0.01

## Matrix with both Scatter + KDE Plots

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.PairGrid(df_train_2[cols], diag_sharey=False)

g.map_lower(sns.scatterplot, s=50)
g.map_lower(corrfunc)
g.map_diag(sns.histplot, kde=True)
g.map_upper(sns.kdeplot, lw=1, cmap="Reds")
g.map_upper(corrfunc)

g.fig.subplots_adjust(wspace=0, hspace=0)

#for ax in g.axes.flat:
#    ax.set_xlabel('')
#    ax.set_ylabel('')

plt.show()

In [ ]:
#sns.set_style("darkgrid")
sns.set_style("dark")


# Top Negative Correlations

In [ ]:
df_train_2[cols]

In [ ]:
print("Top Negative Correlations")
lowest_corrs = get_top_abs_correlations(df_train[numeric_cols], 5, True, False)
lowest_corrs

Collect columns for plotting / comparison later.

In [ ]:
cols = [x[0] for x in lowest_corrs.index]
cols += [x[1] for x in lowest_corrs.index]
cols = list(set(cols))
cols

In [ ]:

#https://stackoverflow.com/questions/30942577/seaborn-correlation-coefficient-on-pairgrid
sns.set(style="white")

g = sns.PairGrid(df_train_2[cols])

g.map_lower(sns.scatterplot, s=50, hue=df_train_2["YearBuilt"]) 
g.map_diag(sns.histplot)
g.map_upper(sns.kdeplot, lw=1, cmap="Reds")
#off diag = off the diagonal = botn upper and lower at the same time
g.map_offdiag(corrfunc)

g.fig.subplots_adjust(wspace=0, hspace=0)

#for ax in g.axes.flat:
#    ax.set_xlabel('')
#    ax.set_ylabel('')

plt.show()

## Correlations Near Zero

The ones that seem most unrelated, sometimes for a reason.

In [ ]:
print("Closest to Zero Correlations")
irr_corrs = get_top_abs_correlations(df_train[numeric_cols], 5, True, True)
irr_corrs

In [ ]:
cols = [x[0] for x in irr_corrs.index]
cols += [x[1] for x in irr_corrs.index]
cols = list(set(cols))
cols

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

g = sns.PairGrid(df_train_2[cols], diag_sharey=False)
#g = sns.PairGrid(df_train_2[cols], diag_sharey=False, hue="YearBuilt")

g.map_lower(sns.scatterplot, s=50,hue=df_train_2["YearBuilt"], palette="BuGn")
g.map_diag(sns.histplot, kde=True)
g.map_lower(corrfunc)
#for some reason kdeplot here does not have any effect with "palette" but "cmap" works
g.map_upper(sns.kdeplot, cmap="BuPu")
g.map_upper(corrfunc)

g.fig.subplots_adjust(wspace=0, hspace=0)

plt.show()

Thats all for today..